In [1]:
import numpy as np
import keras
import pretty_midi
import librosa

import os
import sys
import mir_eval
import glob

import numpy as np
import pandas as pd

sys.path.append("..")
sys.path.append("../src")

%load_ext autoreload
%autoreload 
from src import preprocess
from src import constants
from src import datagen
from src.infer import infer_from_processed
from src.metrics import pm_to_pitches_intervals, get_f1_score_notes, get_f1_score_frames
from src import helpers


Using TensorFlow backend.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
dirname = '/floyd/input/new_maestro/'
preprocess.preprocess(dirname, False)

32


In [2]:
def test_model(model):
    test_generator = datagen.DataGenerator(constants.TEST_PROCESSED_DIR, constants.SEQUENCE_SIZE, constants.BATCH_SIZE, constants.CONTEXT_WINDOW_SIZE, 'x_[0-9]+.npy', 'y_[0-9]+.npy')

    note_f1s = []
    note_precisions = []
    note_recalls = []

    frame_f1s = []
    frame_precisions = []
    frame_recalls = []

    for sample in glob.glob(os.path.join(constants.TEST_PROCESSED_DIR, 'x_*')):
        isolated_filename = sample.split('/')[-1]

        gt_file= os.path.join(constants.TEST_PROCESSED_DIR,test_generator.corresponding_y(isolated_filename))
        referances = np.load(gt_file, mmap_mode='r')
        predictions = infer_from_processed(model, sample)

        padded =  np.zeros((predictions.shape[0], 128))
        padded[:, 21:109] = predictions
        padded =  padded.T
        sequence_est = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]
        padded =  np.zeros((referances.shape[0], 128))
        padded[:, 21:109] = referances
        padded =  padded.T
        sequence_ref = helpers.piano_roll_to_pretty_midi(padded, fs=32, program=0).instruments[0]

        est_intervals,est_pitches = pm_to_pitches_intervals(sequence_est)
        ref_intervals,ref_pitches = pm_to_pitches_intervals(sequence_ref)

        note_precision, note_recall, note_f1 = get_f1_score_notes(ref_intervals, ref_pitches, est_intervals, est_pitches)

        frame_precision, frame_recall, frame_f1 = get_f1_score_frames(referances, predictions)


        note_f1s.append(note_f1)
        note_precisions.append(note_precision)
        note_recalls.append(note_recall)

        frame_f1s.append(frame_f1)
        frame_precisions.append(frame_precision)
        frame_recalls.append(frame_recall)


    # we calculate mean across all scores as done in Ondets and Frames paper for consistency
    avg_note_f1 = np.mean(note_f1s)
    avg_note_precision = np.mean(note_precisions)
    avg_note_recall = np.mean(note_recalls)

    avg_frame_f1 = np.mean(frame_f1s)
    avg_frame_precision = np.mean(frame_precisions)
    avg_frame_recall = np.mean(frame_recalls)

    return {'frame_f1':avg_frame_f1,
            'frame_precision':avg_frame_precision, 
            'frame_recall':avg_frame_recall, 
            'note_f1':avg_note_f1, 
            'note_precision':avg_note_precision, 
            'note_recall':avg_note_recall}


In [3]:
metrics_lstm = test_model('/floyd/home/lstm_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [5]:
lstm_eval = pd.DataFrame({'Precison' : [metrics_lstm['frame_precision'],metrics_lstm['note_precision']], 
                   'Recall' : [metrics_lstm['frame_recall'],metrics_lstm['note_recall']],
                   'F1' : [metrics_lstm['frame_f1'],metrics_lstm['note_f1']],
                    '':['Frame', 'Note']})

lstm_eval.set_index('')

,Precison,Recall,F1
,,,
Frame,0.755206,0.751454,0.750039
Note,0.637897,0.478511,0.540015


In [3]:
metrics_gru = test_model('/floyd/home/gru_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
gru_eval = pd.DataFrame({'Precison' : [metrics_gru['frame_precision'],metrics_gru['note_precision']], 
                   'Recall' : [metrics_gru['frame_recall'],metrics_gru['note_recall']],
                   'F1' : [metrics_gru['frame_f1'],metrics_gru['note_f1']],
                    '':['Frame', 'Note']})


gru_eval.set_index('')

,Precison,Recall,F1
,,,
Frame,0.808718,0.660518,0.706780
Note,0.772961,0.490126,0.589751
